In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install pytorch-lightning  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 585 kB 5.1 MB/s 
     |████████████████████████████████| 596 kB 60.2 MB/s 
     |████████████████████████████████| 419 kB 65.2 MB/s 
     |████████████████████████████████| 140 kB 76.7 MB/s 
     |████████████████████████████████| 1.1 MB 53.7 MB/s 
     |████████████████████████████████| 271 kB 76.7 MB/s 
     |████████████████████████████████| 144 kB 58.2 MB/s 
     |████████████████████████████████| 94 kB 4.4 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
pip install pytorch-forecasting

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 127 kB 5.0 MB/s 
     |████████████████████████████████| 308 kB 33.1 MB/s 
     |████████████████████████████████| 81 kB 10.9 MB/s 
     |████████████████████████████████| 209 kB 72.6 MB/s 
     |████████████████████████████████| 78 kB 9.5 MB/s 
     |████████████████████████████████| 49 kB 7.5 MB/s 
     |████████████████████████████████| 146 kB 62.5 MB/s 
     |████████████████████████████████| 112 kB 75.9 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=f323d325c809d0a40cd5f4320247bd917cc5378c97bf7739bfc8ef6caf37753c
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [4]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [47]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from pytorch_forecasting import Baseline
from pytorch_forecasting.metrics import SMAPE

import joblib
import pickle
import optuna

warnings.filterwarnings('ignore')
%matplotlib inline

### 함수정의

In [6]:
def sliding_windows(data, lookback_length, forecast_length):

    x = []
    y = []
    
    for i in range(lookback_length, len(data) - forecast_length + 1):
        _x = data[(i-lookback_length) : i]
        _y = data[i : (i + forecast_length)]
        x.append(_x)
        y.append(_y)
    return np.array(x), np.array(y)


def get_data_loader(X, y, batch_size):

    x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

    train_ds = TensorDataset(torch.Tensor(x_train), torch.Tensor(y_train))
    train_dl = DataLoader(train_ds, batch_size = batch_size)

    val_ds = TensorDataset(torch.Tensor(x_val), torch.Tensor(y_val))
    val_dl = DataLoader(val_ds, batch_size = batch_size)

    input_size = x_train.shape[-1]

    return train_dl, val_dl, input_size

### Data loading && Preproces

In [7]:
scaler = MinMaxScaler()

data=pd.read_excel('/content/drive/MyDrive/교육/AI실무인증과정/학회 발표/data/data_full.xlsx')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1858 entries, 0 to 1857
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             1858 non-null   datetime64[ns]
 1   Account DOW      1858 non-null   object        
 2   REV OBD          1858 non-null   float64       
 3   OBD NET+FSC_KRW  1858 non-null   float64       
 4   OBD A/R_KRW      1858 non-null   float64       
 5   REV CPN          1858 non-null   float64       
 6   CPN NET+FSC_KRW  1858 non-null   float64       
 7   CPN A/R_KRW      1858 non-null   float64       
 8   REV TKT          1858 non-null   float64       
 9   TKT NET+FSC_KRW  1858 non-null   float64       
 10  TKT A/R_KRW      1858 non-null   float64       
 11  WTI              1858 non-null   float64       
 12  exchanges        1858 non-null   float64       
 13  kospi            1858 non-null   float64       
 14  rates            1858 non-null   float64

In [8]:
res_data = data[data["Date"].isin(pd.date_range('2016-01-04', '2019-11-30'))]
res_data.reset_index(drop=True, inplace=True)
res_data = res_data[['Date', 'REV OBD']]
scale_cols = ['REV OBD']

In [9]:
res_data.shape

(1427, 2)

In [10]:
# 원본 코드 LSTM-pytorch_optuna_05와 다른 부분

# Loockback_period & forecasting_period
max_prediction_length = 20
lookback_length = 100
training_data_max = len(res_data) - max_prediction_length

# 학습용 데이터
data_p = res_data.iloc[:training_data_max, :]
training_data = scaler.fit_transform(data_p[scale_cols])

In [11]:
training_data_max

1407

In [12]:
data_p.shape

(1407, 2)

In [13]:
data_p

,Date,REV OBD
0,2016-01-04,28000.0
1,2016-01-05,24657.0
2,2016-01-06,26920.0
3,2016-01-07,26624.0
4,2016-01-08,28879.0
...,...,...
1402,2019-11-06,25486.0
1403,2019-11-07,25516.0
1404,2019-11-08,30883.0
1405,2019-11-09,27640.0


#### Metric 생성을 위한 oot sample 정의 (원본 코드 LSTM-pytorch_optuna_05와 다른 부분)


In [14]:
# max_prediction_length 만큼의 데이터는 예측 데이터와 비교를 위해 분리
# Training set에 없는 데이터로 구성
# Input과 output의 pair로 정의
x_for_metric = scaler.fit_transform(res_data[training_data_max -lookback_length : training_data_max][scale_cols])
y_for_metric = scaler.fit_transform(res_data[training_data_max:][scale_cols])

In [15]:
x_for_metric.shape

(100, 1)

In [16]:
y_for_metric.shape

(20, 1)

In [17]:
# LSTM은 1 step 뒤의 값만을 예측하므로, forecasting_period를 1로 두고 진행
x, y = sliding_windows(training_data, lookback_length, 1)

### Model 정의

In [18]:
# 원본 코드 LSTM-pytorch_optuna_05와 다른 부분

class LSTM(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size,
                            num_layers = num_layers, batch_first = True)
        
        self.fc = nn.Linear(hidden_size  * num_layers, num_classes)
        
    def forward(self, x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size, device = x.device))
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size, device = x.device))
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        h_out = h_out.view(-1, self.hidden_size * self.num_layers)
        out = self.fc(h_out)
        return out

In [19]:
def train(log_interval, model, train_dl, val_dl, optimizer, criterion, epoch):

    best_loss = np.inf
    for epoch in range(epoch):
        train_loss = 0.0
        model.train()
        for data, target in train_dl:

            if torch.cuda.is_available():
                data, target = data.cuda(), target.cuda()
                model = model.cuda()

            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target) # mean-squared error for regression
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # validation
        valid_loss = 0.0
        model.eval()
        for data, target in val_dl:

            if torch.cuda.is_available():
                data, target = data.cuda(), target.cuda()

            output = model(data)         
            loss = criterion(output, target)
            valid_loss += loss.item()

        if ( epoch % log_interval == 0 ):
            print(f'\n Epoch {epoch} \t Training Loss: {train_loss / len(train_dl)} \t Validation Loss: {valid_loss / len(val_dl)} \n')

        if best_loss > (valid_loss / len(val_dl)):
            print(f'Validation Loss Decreased({best_loss:.6f}--->{(valid_loss / len(val_dl)):.6f}) \t Saving The Model')
            best_loss = (valid_loss / len(val_dl))
            torch.save(model.state_dict(), 'lstm_saved_model.pth')

    return best_loss


def smape(a, f):
    return 1/len(a) * np.sum(2 * np.abs(f-a) / (np.abs(a) + np.abs(f))*100)

In [20]:
# 무슨 의미인가?

aa = x_for_metric
tmp = np.append( np.expand_dims(aa[1:, :], 0), np.expand_dims(y_for_metric[2, :], (0,2)), axis=1)
tmp.shape

(1, 100, 1)

In [21]:
def objective(trial):

    cfg = { 
            'batch_size' : trial.suggest_categorical('batch_size',[32, 64, 128, 256, 512]), # [64, 128, 256],
            'num_epochs' : trial.suggest_int('num_epochs', 5, 50, 5),
            'learning_rate' : trial.suggest_loguniform('learning_rate', 1e-3, 1e-1), #trial.suggest_loguniform('learning_rate', 1e-2, 1e-1), # learning rate을 0.01-0.1까지 로그 uniform 분포로 사용
            'hidden_size': trial.suggest_categorical('hidden_size',[32, 64, 128, 256, 512]),
            'num_layers': trial.suggest_int('num_layers', 1, 7, 1),       
        }

    torch.manual_seed(42)

    log_interval = 5
    num_classes = 1 # parameter에서 빼서 상수로 설정

    train_dl, val_dl, input_size = get_data_loader(x, y,  cfg['batch_size'])
    
    model = LSTM(
        num_classes = num_classes, 
        input_size = input_size, 
        hidden_size = cfg['hidden_size'], 
        num_layers = cfg['num_layers']
    )
    
    if torch.cuda.is_available():
        model = model.cuda()
        
    optimizer = optim.Adam(model.parameters(), lr=cfg['learning_rate'])
    criterion = torch.nn.MSELoss()
    best_loss = train(log_interval, model, train_dl, val_dl, optimizer, criterion, cfg['num_epochs'])

    print('best loss for the trial = ', best_loss)
    predict_data = []
    # 여기서 x는 (sample, lookback_length, 1)의 크기를 지님. 따라서, 제일 앞의 시점을 제거하려면, x[:, -1, :]이 되어야 함
    x_pred = np.expand_dims(x_for_metric, 0)  # Inference에 사용할 lookback data를 x_pred로 지정. 앞으로 x_pred를 하나씩 옮겨 가면서 inference를 할 예정

    for j, i in enumerate(range(max_prediction_length)):

        # feed the last forecast back to the model as an input
        x_pred = np.append( x_pred[:, 1:, :], np.expand_dims(y_for_metric[j, :], (0,2)), axis=1)
        xt_pred = torch.Tensor(x_pred)

        if torch.cuda.is_available():
            xt_pred = xt_pred.cuda()
        # generate the next forecast
        yt_pred = model(xt_pred)
        # tensor to array
        # x_pred = xt_pred.cpu().detach().numpy()
        y_pred = yt_pred.cpu().detach().numpy()

        # save the forecast
        predict_data.append(y_pred)

    # transform the forecasts back to the original scale
    predict_data = np.array(predict_data).reshape(-1, 1)
    SMAPE = smape(y_for_metric, predict_data)
    
    print(f' \nSMAPE : {SMAPE}')


    return SMAPE

In [ ]:
sampler = optuna.samplers.TPESampler()
#   sampler = optuna.samplers.SkoptSampler()

# model.load_state_dict(torch.load('lstm_saved_model.pth'))
    
study = optuna.create_study(sampler=sampler, direction='minimize')
study.optimize(objective, n_trials= 7)
joblib.dump(study, '/content/drive/MyDrive/교육/AI실무인증과정/LSTM/LSTM_pytorch_optuna_05.pkl')

[I 2022-06-18 23:58:24,273] A new study created in memory with name: no-name-ec936d39-d544-4282-bd6d-1490de67ebd4



 Epoch 0 	 Training Loss: 0.038367086254498535 	 Validation Loss: 0.01904766969382763 

Validation Loss Decreased(inf--->0.019048) 	 Saving The Model
Validation Loss Decreased(0.019048--->0.018740) 	 Saving The Model

 Epoch 5 	 Training Loss: 0.023578424335402602 	 Validation Loss: 0.01914496049284935 


 Epoch 10 	 Training Loss: 0.02353679827030967 	 Validation Loss: 0.01885393410921097 

Validation Loss Decreased(0.018740--->0.018725) 	 Saving The Model

 Epoch 15 	 Training Loss: 0.02332561063196729 	 Validation Loss: 0.01869758479297161 

Validation Loss Decreased(0.018725--->0.018698) 	 Saving The Model
Validation Loss Decreased(0.018698--->0.018672) 	 Saving The Model
Validation Loss Decreased(0.018672--->0.018648) 	 Saving The Model
Validation Loss Decreased(0.018648--->0.018626) 	 Saving The Model
Validation Loss Decreased(0.018626--->0.018605) 	 Saving The Model

 Epoch 20 	 Training Loss: 0.02315993530347067 	 Validation Loss: 0.018586138635873793 

Validation Loss Decreas

[I 2022-06-18 23:58:31,366] Trial 0 finished with value: 63.670803759803135 and parameters: {'batch_size': 64, 'num_epochs': 25, 'learning_rate': 0.002044602538075323, 'hidden_size': 64, 'num_layers': 3}. Best is trial 0 with value: 63.670803759803135.


Validation Loss Decreased(0.018535--->0.018521) 	 Saving The Model
best loss for the trial =  0.018520670756697655
 
SMAPE : 63.670803759803135

 Epoch 0 	 Training Loss: 0.10708567603594726 	 Validation Loss: 0.04734290887912115 

Validation Loss Decreased(inf--->0.047343) 	 Saving The Model
Validation Loss Decreased(0.047343--->0.018484) 	 Saving The Model
Validation Loss Decreased(0.018484--->0.017930) 	 Saving The Model

 Epoch 5 	 Training Loss: 0.02333067274755902 	 Validation Loss: 0.018031633148590725 


 Epoch 10 	 Training Loss: 0.023188776440090604 	 Validation Loss: 0.018173179278771084 


 Epoch 15 	 Training Loss: 0.02323819138109684 	 Validation Loss: 0.01823676998416583 


 Epoch 20 	 Training Loss: 0.023237262955970235 	 Validation Loss: 0.018232546746730804 


 Epoch 25 	 Training Loss: 0.023207451527317364 	 Validation Loss: 0.018209465468923252 



[I 2022-06-18 23:58:38,950] Trial 1 finished with value: 69.24156728626375 and parameters: {'batch_size': 128, 'num_epochs': 30, 'learning_rate': 0.01732550223490845, 'hidden_size': 32, 'num_layers': 5}. Best is trial 0 with value: 63.670803759803135.


...

[I 2022-06-18 23:59:10,221] Trial 3 finished with value: 61.68390990969686 and parameters: {'batch_size': 64, 'num_epochs': 30, 'learning_rate': 0.0018782118513314694, 'hidden_size': 32, 'num_layers': 1}. Best is trial 3 with value: 61.68390990969686.


best loss for the trial =  0.017907843738794327
 
SMAPE : 61.68390990969686

 Epoch 0 	 Training Loss: 0.031122662127017975 	 Validation Loss: 0.02232905700802803 

Validation Loss Decreased(inf--->0.022329) 	 Saving The Model
Validation Loss Decreased(0.022329--->0.020016) 	 Saving The Model
Validation Loss Decreased(0.020016--->0.018417) 	 Saving The Model

 Epoch 5 	 Training Loss: 0.0234923873863676 	 Validation Loss: 0.01881374940276146 


 Epoch 10 	 Training Loss: 0.023327343385009205 	 Validation Loss: 0.01873388811945915 


 Epoch 15 	 Training Loss: 0.023238001719993705 	 Validation Loss: 0.018687807023525238 


 Epoch 20 	 Training Loss: 0.023207464364959914 	 Validation Loss: 0.018733027577400207 


 Epoch 25 	 Training Loss: 0.023540742092711085 	 Validation Loss: 0.018918886408209802 



[I 2022-06-18 23:59:14,326] Trial 4 finished with value: 62.94609254921562 and parameters: {'batch_size': 64, 'num_epochs': 30, 'learning_rate': 0.028659953568405217, 'hidden_size': 64, 'num_layers': 1}. Best is trial 3 with value: 61.68390990969686.


best loss for the trial =  0.0184166606515646
 
SMAPE : 62.94609254921562


['/content/drive/MyDrive/교육/AI실무인증과정/LSTM/LSTM_pytorch_optuna_05.pkl']

In [22]:
study = joblib.load('/content/drive/MyDrive/교육/AI실무인증과정/LSTM/LSTM_pytorch_optuna_05.pkl') 
df = study.trials_dataframe() 
df.sort_values('value')

,number,value,datetime_start,datetime_complete,duration,params_batch_size,params_hidden_size,params_learning_rate,params_num_epochs,params_num_layers,state
3,3,61.683910,2022-06-18 23:59:06.503369,2022-06-18 23:59:10.220675,0 days 00:00:03.717306,64,32,0.001878,30,1,COMPLETE
4,4,62.946093,2022-06-18 23:59:10.224759,2022-06-18 23:59:14.325827,0 days 00:00:04.101068,64,64,0.028660,30,1,COMPLETE
0,0,63.670804,2022-06-18 23:58:24.279572,2022-06-18 23:58:31.366290,0 days 00:00:07.086718,64,64,0.002045,25,3,COMPLETE
1,1,69.241567,2022-06-18 23:58:31.368762,2022-06-18 23:58:38.950067,0 days 00:00:07.581305,128,32,0.017326,30,5,COMPLETE
2,2,200.000000,2022-06-18 23:58:38.952322,2022-06-18 23:59:06.501205,0 days 00:00:27.548883,32,256,0.065448,25,4,COMPLETE


In [23]:
study.best_params

{'batch_size': 64,
 'hidden_size': 32,
 'learning_rate': 0.0018782118513314694,
 'num_epochs': 30,
 'num_layers': 1}

In [24]:
optuna.visualization.plot_param_importances(study)

In [25]:
optuna.visualization.plot_optimization_history(study)

In [40]:
torch.manual_seed(42)

log_interval = 5
num_classes = 1 # parameter에서 빼서 상수로 설정

train_dl, val_dl, input_size = get_data_loader(x, y,  study.best_params['batch_size'])

model = LSTM(
    num_classes = num_classes, 
    input_size = input_size, 
    hidden_size = study.best_params['hidden_size'], 
    num_layers = study.best_params['num_layers']
)

if torch.cuda.is_available():
    model = model.cuda()
    
optimizer = optim.Adam(model.parameters(), lr=study.best_params['learning_rate'])
criterion = torch.nn.MSELoss()
best_loss = train(log_interval, model, train_dl, val_dl, optimizer, criterion, study.best_params['num_epochs'])

print('best loss for the trial = ', best_loss)
predict_data = []
# 여기서 x는 (sample, lookback_length, 1)의 크기를 지님. 따라서, 제일 앞의 시점을 제거하려면, x[:, -1, :]이 되어야 함
x_pred = np.expand_dims(x_for_metric, 0)  # Inference에 사용할 lookback data를 x_pred로 지정. 앞으로 x_pred를 하나씩 옮겨 가면서 inference를 할 예정

for j, i in enumerate(range(max_prediction_length)):

    # feed the last forecast back to the model as an input
    x_pred = np.append( x_pred[:, 1:, :], np.expand_dims(y_for_metric[j, :], (0,2)), axis=1)
    xt_pred = torch.Tensor(x_pred)

    if torch.cuda.is_available():
        xt_pred = xt_pred.cuda()
    # generate the next forecast
    yt_pred = model(xt_pred)
    # tensor to array
    # x_pred = xt_pred.cpu().detach().numpy()
    y_pred = yt_pred.cpu().detach().numpy()

    # save the forecast
    predict_data.append(y_pred)

# transform the forecasts back to the original scale
predict_data = np.array(predict_data).reshape(-1, 1)
SMAPE = smape(y_for_metric, predict_data)

print(f' \nSMAPE : {SMAPE}')


 Epoch 0 	 Training Loss: 0.09048183492439635 	 Validation Loss: 0.021324257552623748 

Validation Loss Decreased(inf--->0.021324) 	 Saving The Model
Validation Loss Decreased(0.021324--->0.018006) 	 Saving The Model
Validation Loss Decreased(0.018006--->0.017908) 	 Saving The Model

 Epoch 5 	 Training Loss: 0.022353264338829937 	 Validation Loss: 0.01806236766278744 


 Epoch 10 	 Training Loss: 0.022442576530225137 	 Validation Loss: 0.018069243617355825 


 Epoch 15 	 Training Loss: 0.022439642042359886 	 Validation Loss: 0.018061178363859655 


 Epoch 20 	 Training Loss: 0.02242254980785005 	 Validation Loss: 0.0180482380092144 


 Epoch 25 	 Training Loss: 0.02240299230770153 	 Validation Loss: 0.01803500484675169 

best loss for the trial =  0.01790784318000078
 
SMAPE : 61.68390938017316


In [70]:
# actual_data는 max_prediction_length에 해당하는 20일의 기간 : 실제 예측하려는 기간 (2019년 12월1일 ~ 2019년 12월 20일)
actual_data = data[data["Date"].isin(pd.date_range('2019-12-01', '2019-12-20'))]
actual_data['time_index'] = np.arange(len(actual_data))
actual_data['market'] = 'OBD'
actual_data.head()

,Date,Account DOW,REV OBD,OBD NET+FSC_KRW,OBD A/R_KRW,REV CPN,CPN NET+FSC_KRW,CPN A/R_KRW,REV TKT,TKT NET+FSC_KRW,TKT A/R_KRW,WTI,exchanges,kospi,rates,stock_a,stock_k,stock_kkj,time_index,market
1430,2019-12-01,SUN,28090.0,1.093871e+10,389416.4866,10669.0,3.644840e+09,341629.0611,5973.0,3.888420e+09,650999.5691,55.17,1180.0,2087.96,1.39,25391.0,16048.0,47426.0,0,OBD
1431,2019-12-02,MON,27057.0,9.887493e+09,365431.9919,40638.0,1.622895e+10,399354.0099,21846.0,1.747528e+10,799930.3397,55.96,1185.0,2091.92,1.43,24770.0,16048.0,47377.0,1,OBD
1432,2019-12-03,TUE,22931.0,7.988729e+09,348381.2026,37530.0,1.512760e+10,403080.2320,20447.0,1.633829e+10,799055.6766,56.10,1190.0,2084.07,1.49,24818.0,15854.0,46009.0,2,OBD
1433,2019-12-04,WED,23651.0,8.167825e+09,345347.9761,36150.0,1.466185e+10,405583.5527,19725.0,1.582003e+10,802029.2492,58.43,1193.0,2068.89,1.41,24436.0,15724.0,45618.0,3,OBD
1434,2019-12-05,THU,25570.0,9.347876e+09,365579.8221,37518.0,1.617110e+10,431022.5730,20295.0,1.755050e+10,864769.6796,58.43,1191.0,2060.74,1.42,24245.0,15919.0,46107.0,4,OBD


In [71]:
# 실제 예측하려는 target prediction 구간 : 20일
actuals = actual_data['REV OBD']
actuals

1430    28090.0
1431    27057.0
1432    22931.0
1433    23651.0
1434    25570.0
1435    29559.0
1436    26113.0
1437    28809.0
1438    27316.0
1439    23017.0
1440    25317.0
1441    25544.0
1442    29799.0
1443    27079.0
1444    29413.0
1445    29138.0
1446    24957.0
1447    28161.0
1448    27917.0
1449    32165.0
Name: REV OBD, dtype: float64

In [72]:
actuals = torch.tensor(actual_data['REV OBD'].values)
print(actuals)
print(actuals.shape)

tensor([28090., 27057., 22931., 23651., 25570., 29559., 26113., 28809., 27316.,
        23017., 25317., 25544., 29799., 27079., 29413., 29138., 24957., 28161.,
        27917., 32165.], dtype=torch.float64)
torch.Size([20])


In [73]:
baseline_predictions =  torch.tensor([data.loc[data.Date == '2019-11-30']['REV OBD'].values[0]] * len(actuals)).to(torch.float32)

mae_with_baseline = (actuals - baseline_predictions).abs().mean().item()
smape_with_baseline = (2 * (baseline_predictions - actuals).abs() / (baseline_predictions.abs() + actuals.abs() + 1e-8)).mean()

print(f'mae_with_baseline : {mae_with_baseline}')
print(f'smape_with_baseline : {smape_with_baseline}')

mae_with_baseline : 2287.35
smape_with_baseline : 0.08517351399239322


In [74]:
predicts = scaler.inverse_transform(predict_data)
print(predicts.shape)
print(predicts)
print(type(predicts))

(20, 1)
[[25832.44 ]
 [25626.494]
 [25634.81 ]
 [25668.057]
 [25935.832]
 [26046.277]
 [26251.129]
 [26332.691]
 [25992.012]
 [25931.81 ]
 [25909.61 ]
 [26140.588]
 [26147.004]
 [26231.732]
 [26279.799]
 [25921.818]
 [25768.852]
 [25659.232]
 [25864.012]
 [25833.652]]
<class 'numpy.ndarray'>


In [75]:
predicts = torch.tensor(predicts)
predicts.shape

torch.Size([20, 1])

In [76]:
(2 * (predicts - actuals).abs() / (predicts.abs() + actuals.abs() + 1e-8)).mean()

tensor(0.0818, dtype=torch.float64)

In [77]:
smape(actuals.numpy(), predicts.numpy())

163.6149575834667

In [68]:
mae_with_test = (actuals - predicts).abs().mean().item()
smape_with_test = smape(actuals, predicts)

print(f'mae_with_test : {mae_with_test}')
print(f'smape_with_test : {smape_with_test}')

TypeError: ignored

In [38]:
mae_with_test = (actuals - predicts).abs().mean().item()
mae_with_test 

2143.826171875

In [ ]:
y_train.shape

np.expand_dims(pred, axis=1).shape

predicted_for_plot = sc.inverse_transform(np.concatenate((y_train, np.expand_dims(pred, axis=1))))
actual_for_plot = sc.inverse_transform(np.concatenate((y_train, y_test)))

predicted_for_plot.shape

actual_for_plot.shape

plt.plot(predicted_for_plot[300:])
plt.plot(actual_for_plot[300:])
plt.suptitle('Timeseries Prediction')
plt.axvline(x=y_train.shape[0]-300, c='r', linestyle='--')
plt.show()

NameError: ignored